In [1]:
import SimpleITK as sitk
import numpy as np

images = []

for num in [915, 925, 935, 945, 955, 995]:
    img = sitk.ReadImage(f'./Heart/Predicted_NIfTI_masks/heart_{num}.nii.gz')
    arr = sitk.GetArrayFromImage(img)
    images.append(arr)
print("Finished Loading Images")

print("\nSanity size Check")
for i in images:
    print(i.shape)

Finished Loading Images

Sanity size Check
(256, 512, 512)
(256, 512, 512)
(256, 512, 512)
(256, 512, 512)
(256, 512, 512)
(256, 512, 512)


In [8]:
import fbx
from skimage import measure
import nibabel as nib
from scipy.ndimage import gaussian_filter

# Load the NIfTI file
voxel_data = images[0].astype(int)
# voxel_data = transformed_img_array

# Can choose to apply gaussian filter
# smoothed_data = gaussian_filter(voxel_data, sigma=1)  # Adjust sigma for smoothing intensity
# verts, faces, normals, _ = measure.marching_cubes(smoothed_data, level=0.3)
verts, faces, normals, _ = measure.marching_cubes(voxel_data, level=0.3)
print(f"Original model has {len(verts)} vertices and {len(faces)} faces")

Original model has 1214369 vertices and 2426072 faces


In [13]:
# === Step 3: Mesh Optimization Algorithm ===
def compute_energy(verts, faces, original_verts, alpha=1.0, beta=0.1):
    """
    Compute the total energy: Data fidelity + Compactness.
    """
    # Data fidelity term
    fidelity_loss = np.sum(np.linalg.norm(verts - original_verts, axis=1)**2)

    # Compactness term: Penalize the number of vertices and faces
    compactness_loss = len(verts) + len(faces)

    # Total energy
    return alpha * fidelity_loss + beta * compactness_loss

def optimize_mesh(verts, faces, voxel_data, iterations=50, lr=0.01, alpha=1.0, beta=0.1):
    """
    Optimize the mesh to minimize the energy function.
    """
    verts = verts.copy()
    original_verts = verts.copy()

    for i in range(iterations):
        # Compute energy
        energy = compute_energy(verts, faces, original_verts, alpha, beta)
        
        # Compute gradients (example based on fidelity loss)
        fidelity_grad = 2 * (verts - original_verts)  # Simplified fidelity gradient
        compactness_grad = np.zeros_like(verts)  # Compactness doesn't directly affect vertex positions
        
        # Total gradient
        total_grad = fidelity_grad + compactness_grad

        # Update vertex positions
        verts -= lr * total_grad

        # Log progress
        print(f"Iteration {i + 1}, Energy: {energy:.4f}")

        # Early stopping if energy change is negligible
        if i > 0 and abs(prev_energy - energy) < 1e-6:
            print("Converged.")
            break

        prev_energy = energy

    return verts

# Optimize the mesh
optimized_verts = optimize_mesh(verts, faces, voxel_data, iterations=100, lr=0.5, alpha=1.0, beta=0.2)


Iteration 1, Energy: 728088.1875
Iteration 2, Energy: 728088.1875
Converged.


In [ ]:
# === Step 4: Export to FBX ===
# Initialize the FBX SDK
manager = fbx.FbxManager.Create()
scene = fbx.FbxScene.Create(manager, "OptimizedMeshScene")

# Create a mesh object
mesh = fbx.FbxMesh.Create(scene, "OptimizedMesh")

# Set the vertices in the FBX mesh
mesh.InitControlPoints(len(optimized_verts))
for i, vert in enumerate(optimized_verts):
    mesh.SetControlPointAt(fbx.FbxVector4(*vert), i)

# Add polygons to the mesh
for face in faces:
    mesh.BeginPolygon()
    for idx in face:
        mesh.AddPolygon(idx)
    mesh.EndPolygon()

# Create a node for the mesh
mesh_node = fbx.FbxNode.Create(scene, "OptimizedMeshNode")
mesh_node.SetNodeAttribute(mesh)

# Add the mesh node to the scene
scene.GetRootNode().AddChild(mesh_node)

# Export the scene to an FBX file
exporter = fbx.FbxExporter.Create(manager, "")
output_fbx_path = "Optimized_Heart_Mesh.fbx"
exporter.Initialize(output_fbx_path, -1, manager.GetIOSettings())
exporter.Export(scene)
exporter.Destroy()
manager.Destroy()
print("FBX file has been successfully created!")

# === Step 5: File Size Check ===
if os.path.exists(output_fbx_path):
    file_size_bytes = os.path.getsize(output_fbx_path)
    file_size_mb = file_size_bytes / (1024 * 1024)  # Convert bytes to MB
    print(f"File size: {file_size_mb:.2f} MB")
else:
    print("File not found")
